This notebook is meant to serve as an example walk through of the REST API.

For demonstration purposes, we use the secret token we use to run tests. YOU SHOULD USE YOUR OWN TEAMS SECRET.

If you do not have your `Team Secret Key`, send a slack message to `Alice Yepremyan` or email to `alice.r.yepremyan@jpl.nasa.gov`


In [ ]:
import requests
import os
import json

In [ ]:
secret = os.environ.get('TEST_SECRET')
# url = 'http://localhost:5000'
url = 'https://api-dev.lollllz.com'

# List all of the available problems

### Note that for your development purposes this is a different list than what will be exposed during eval time. During eval time, you will be expected to go through all of the problems this returns, either sequentially or in parallel.

NOTE: We pass the environment variable `GOVTEAM_SECRET` through here in the headers. During prototyping of TA1 systems this will be empty and not matter, during evaluation, the govteam will inject this enviornment variable into your containers and is REQUIRED for hitting the eval endpoint.

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/list_tasks", headers=headers)
r.json()

*NOTE* 
There are two tasks, `problem_test_image_classification`, `problem_test_obj_detection` that list their problem types in the name. These are given for demonstration and testing purposes in this notebook. We also include them in the `dev`, `staging`, and `eval` endpoints to run unit tests against, but they are not true evaluation tasks. True evaluation tasks are named with a uuid as in the other development tasks you see listed. You can find the tasks' problem types listed in their metadata.
For an *evaluation*, you should *filter out* these two testing problem types as shown below, but do not hard code the number of tasks your program runs as these are not guaranteed to be in order. 
(It will not count against you if you simply run all of the tasks, but you may not wish to waste compute time on these tasks.)

In [ ]:
tasks = [task for task in r.json()['tasks'] if task not in ['problem_test_image_classification',
  'problem_test_obj_detection', 'problem_test_video_classification']]

# Get A Task's Metadata

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/task_metadata/problem_test_image_classification", headers=headers)
print(json.dumps(r.json(), indent=2))

You can also get metadata on each whitelisted dataset like so:

In [ ]:
r.json()['task_metadata']['whitelist']

`domain_net-painting` is in the whitelist, so we will get its metadata:

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/dataset_metadata/domain_net-painting", headers=headers)
print(json.dumps(r.json(), indent=2))

We will get all of the tasks of each type. 

In [ ]:
# Helper function to get all tasks of a particular type
from typing import List
def get_task_subset_by_type(subset_type: str, url: str) -> List[str]:
    """
    Helper function that returns the task ids in a list that match a specified
    problem type
    
    Params
    ------
    
    subset_type : str
        The task_type subset you want to get back
    """
    headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    tasks = requests.get(f"{url}/list_tasks", headers=headers)
    task_list = tasks.json()['tasks']
    subset_tasks = []
    for _task in task_list:
        r = requests.get(f"{url}/task_metadata/{_task}", headers=headers)
        task_metadata = r.json()
        try:
            if task_metadata['task_metadata']['problem_type'] == subset_type:
                subset_tasks.append(_task)
        except Exception as e:
            print(_task)
            print(e)
    return subset_tasks

In [ ]:
img_classification_tasks = get_task_subset_by_type('image_classification', url)
obj_detection_tasks = get_task_subset_by_type('object_detection', url)
machine_translation_tasks = get_task_subset_by_type('machine_translation', url)
video_classification_tasks = get_task_subset_by_type('video_classification', url)

In [ ]:
img_classification_tasks

Remember to filter out `problem_test_image_classification` in the evaluation. 

In [ ]:
obj_detection_tasks

Remember to filter out `problem_test_obj_detection` in the evaluation. 

In [ ]:
machine_translation_tasks

Remember to filter out `problem_test_video_classification` in the evaluation.

In [ ]:
video_classification_tasks

## Download the Task Training and Test Data
Now that you see what the task dataset names are, you can use the download.py script from the [dataset_prep repo](https://gitlab.lollllz.com/lwll/dataset_prep). 


In this example walkthrough, the first task we will be using is `problem_test_image_classification`, so we would download the base and adaptation datasets listed in the metadata. In this case both the base and adaptation in `mnist`, though that will usually not be the case. 

`python download.py --dataset mnist --stage development --output ~/lwll_datasets --overwrite True`

# Create a New Session

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
data_type = 'sample' # can either be `sample` or `full`

# Option to customize the session name 
r = requests.post(f"{url}/auth/create_session", json={'session_name': 'testing', 'data_type': data_type, 
                                                      'task_id': 'problem_test_image_classification'},
                  headers=headers)
r.json()

In [ ]:
session_token = r.json()['session_token']

In [ ]:
session_token

# View the Current Session's Metadata at any time

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

In [ ]:
r = requests.get(f"{url}/get_seen_labels", headers=headers)
r.json()

# View the List of Active Sessions at any time

In [ ]:
#This shows the active sessions for your team
headers_session = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/list_active_sessions", headers=headers_session)
active_sessions = r.json()
active_sessions

# Deactivate Sessions

This shows how to progromatticaly deactivate a session. We comment this out for now to show functionality across the rest of the workflow

In [ ]:
#Let's deactivate the last used session
# deactivate_session = active_sessions['active_sessions'][-1]
# headers_active_session = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# r = requests.post(f"{url}/deactivate_session", json={'session_token': deactivate_session}, headers=headers_active_session)
# print(r.json())

In [ ]:
# r = requests.get(f"{url}/session_status", headers=headers_active_session)
# r.json()

In [ ]:
#If len(active_sessions['active_sessions']) was 1, then run this cell to create a new session again.
#Since we don't have any sessions active now, we need to create a new one
# headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# # This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
# data_type = 'sample' # can either be `sample` or `full`

# # Option to customize the session name 
# r = requests.post(f"{url}/auth/create_session", json={'session_name': 'testing', 'data_type': data_type, 'task_id': tasks[0]}, headers=headers)
# session_token = r.json()['session_token']

# Skip Checkpoints
Performers are able to skip checkpoints BUT you may NOT skip the last checkpoint in the base and adaptation stages respectively (ie checkpoint 8 and checkpoint 16). This will result in an exception ERROR. Moreover you should NOT skip more than 2 checkpoints. Below is an example for how to skip a checkpoint.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/skip_checkpoint", headers=headers)
r.json()

# Get Seed Labels
Seed labels may be requested for the first 4 checkpoints, which have label budgets of 1, 2, 4, 8 per class. Note that the `secondary_seed_labels` endpoint has been removed, and you may instead call `seed_labels` repeatedly.

*Note* that in the event you try calling `seed_labels` outside of the appropriate checkpoint, you will encounter an error. This route is only available for the call on the correct checkpoint stage and it will automatically reduce the `budget_left_until_checkpoint` variable. The tasks have been designed such that after calling the `seed_labels` route, you will be exactly at the correct number to submit predictions.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
r.json()

Our budget for the first checkpoint is one label per class, in this case 10, which we have reached. Let's take a look at the Session Metadata to see that we have our `budget_left_until_checkpoint` equal to 0 now. This means we must submit predictions for our holdout test set now.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

# Submitting Predictions

Each dataset is budgeted into 8 checkpoints for base and adaptation. We have just completed the first stage of requesting labels, and we must submit predictions. In this notebook we will show the two supported methods for submitting predictions. Below we will simply generate random predictions to demonstrate how to submit.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Tuple, List
import random

In [ ]:
# My path to my predownloaded datasets by using the download.py utility found 
# here: https://gitlab.lollllz.com/lwll/dataset_prep
# DATASETS_PATH = Path.home().joinpath('lwll_datasets/development')
DATASETS_PATH = Path.home() / "Documents" / "LWLL" / "lwll_datasets" / "development"

In [ ]:
def get_test_images_and_classes(dataset_path: Path, session_token: str, data_type: str='sample') -> Tuple[List[str],List[str]]:
    """
    Helper method to dynamically get the test labels and give us the possible classes that can be submitted
    for the current dataset
    
    Params
    ------
    
    dataset_path : Path
        The path to the `development` dataset downloads
    
    session_token : str
        Your current session token so that we can look up the current session metadata
    
    data_type: str
        Indicates whether you are using the `sample` or `full` dataset. 
    Returns
    -------
    
    Tuple[List[str], List[str]]
        The list of test image ids needed to submit a prediction and the list of class names that you can predict against
    """
    # Then we can just reference our current metadata to get our dataset name and use that in the path
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    r = requests.get(f"{url}/session_status", headers=headers)
    current_dataset = r.json()['Session_Status']['current_dataset']
    current_dataset_name = current_dataset['name']
    current_dataset_classes = current_dataset['classes']

    test_imgs_dir = dataset_path.joinpath(f"{current_dataset_name}/{current_dataset_name}_{data_type}/test")
    test_imgs = [f.name for f in test_imgs_dir.iterdir() if f.is_file()]
    return test_imgs, current_dataset_classes

def generate_random_predictions_on_test_set(test_imgs: List[str], current_dataset_classes: List[str]) -> pd.DataFrame:
    """
    Generates a prediction dataframe for image classification based on random sampling from our available classes
    
    example:
    
    ========  =====
    id        class
    ========  =====
    6831.png  '3'
    1186.png  '9'
    8149.png  '6'
    4773.png  '3'
    3752.png  '10'
    ========  =====
    """
    rand_lbls = [str(random.choice(current_dataset_classes)) for _ in range(len(test_imgs))]
    df = pd.DataFrame({'id': test_imgs, 'class': rand_lbls})
    return df

In [ ]:
test_imgs, current_dataset_classes = get_test_images_and_classes(DATASETS_PATH, session_token)

In [ ]:
# We can use our looked up test image ids along with random sampling from the available classes to make our
# prediction DataFrame
df = generate_random_predictions_on_test_set(test_imgs, current_dataset_classes)

In [ ]:
df.head()

As predictions submission requires multiple, computationally expensive metrics to be calculated, it can occassionally be slow to wait for a response. For this reason, there are two ways we can submit using the `{url}/submit_predictions?metrics` flag. The default way if you don't explicitly set this flag is `{url}/submit_predictions?metrics=true` and will return just the `accuracy` metric. If you change the flag to `{url}/submit_predictions?metrics=false` though, no metrics will be returned and the endpoint should return much faster. Regardless of how the flag is set though, on the last endpoint, all metrics will be returned.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

##### Example Response with f"{url}/submit_predictions?metrics=true"
```
{
    'Session_Status': {
        'accuracy': 0.123,
        'active': 'In Progress',
        'session_name': 'testing',
        'session_token': 'token',
        'task_id': 'task_id',
        'uid': 'uid',
        'user_name': 'name'
    }
}
```
##### Example Response with f"{url}/submit_predictions?metrics=false"
```
{
    'Session_Status': {
        'session_token': 'token'
    }
}
```

In the eval1 version of the api, we could get a second round of seed labels from `secondary_seed_labels`. This endpoint has been removed, and we simply call `seed_labels` again. It will return an additional example from each class, so that at the second checkpoint we have a total of 2 examples per class. You could also choose to stop calling `seed_labels`, and instead query for 10 labels of your choosing. For this example, we will get a second set of seed labels. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
r.json()

Exactly as in the first checkpoint, if we inspect the Session Metadata, we will see our `budget_left_until_checkpoint` equal to 0 now. This means we must submit predictions for our holdout test set again for our second checkpoint.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

In [ ]:
test_imgs, current_dataset_classes = get_test_images_and_classes(DATASETS_PATH, session_token)

In [ ]:
df = generate_random_predictions_on_test_set(test_imgs, current_dataset_classes)

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

Now we are at the third checkpoint. Once again, you can choose to call the `seed_labels` endpoint to be given an additional 2 examples per class or to query for labels. We will now request some labels by id, and then attempt to call `seed_labels` again to demonstrate a potential error. 

# Request Labels

We can request labels up until we exhaust out `budget_left_until_checkpoint`. You can request labels by id at *any* of the checkpoints, including the first four if you have not already requested seed labels for that checkpoint. 

**Note: If you request less than the `budget_left_until_checkpoint` out of the dataset and then submit predictions, your session will look as if you requested all labels for that checkpoint. Because of this it is to your advantange to request as many labels as you have available to you or else you are going into future checkpoints with less information**

In [ ]:
def get_random_labels_from_train_dataset(dataset_path: Path, session_token: str, n: int=None, data_type: str='sample') -> List[str]:
    """
    Helper function to get a random `n` image ids from our train dataset to request labels for
    from the api
    
    Params
    ------
    
    dataset_path : Path
        The path to the `development` dataset downloads
    
    session_token : str
        Your current session token so that we can look up the current session metadata
    data_type: str
        Indicates whether you are using the `sample` or `full` size dataset
    
    Returns
    -------
    
    List[str]
        A list of n unique image ids for the current session dataset
        
    """
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    r = requests.get(f"{url}/session_status", headers=headers)
    current_dataset = r.json()['Session_Status']['current_dataset']
    current_dataset_name = current_dataset['name']
    budget_left = r.json()['Session_Status']['budget_left_until_checkpoint'] 
    if not n:
        n = budget_left
        print(f"budget_left is {budget_left}")
        
    train_imgs_dir = dataset_path.joinpath(f"{current_dataset_name}/{current_dataset_name}_{data_type}/train")
    train_imgs = [f.name for f in train_imgs_dir.iterdir() if f.is_file()]
    random_ids = random.sample(train_imgs, k=n)
    return random_ids
    

There are 20 labels left in our budget, but for illustration, we will first request 10. 

In [ ]:
images_to_be_labeled = get_random_labels_from_train_dataset(DATASETS_PATH, session_token, n=10)
images_to_be_labeled

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

query = {
    'example_ids': images_to_be_labeled
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
r.json()

Now our `budget_left_until_checkpoint` is 10. Note that now that we have started to query for labels by id at this checkpoint, we cannot get `seed_labels` for the remainder of our budget, we must have the entire budget for the checkpoint available if we want to get `seed_labels`. If we call `seed_labels` again, we will get an error. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
r.json()

We will query by id again until we run out of `budget_left_until_checkpoint`. Then we have to submit predictions to advance.

In [ ]:
images_to_be_labeled = get_random_labels_from_train_dataset(DATASETS_PATH, session_token, n=10)
images_to_be_labeled

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

query = {
    'example_ids': images_to_be_labeled
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
r.json()

Now we have 0 labels left in our budget, and we must submit our predictions. 

In [ ]:
df = generate_random_predictions_on_test_set(test_imgs, current_dataset_classes)
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

Now we are on the fourth checkpoint, and we may request `seed_labels` again for an additional four examples per class. We will request seed labels again for demonstration, but it may not be to your advantage to request seed labels after querying by id because seed labels are chosen deterministically in advance for each task, so you may get the same label as one of the ones you requested by id. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
r.json()

Our session status will show that we have used all of our budget for this checkpoint:

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

In [ ]:
# Submit predictions
df = generate_random_predictions_on_test_set(test_imgs, current_dataset_classes)
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

Now we are at the fifth checkpoint. The 5th through 8th checkpoint budget labels are on a logarithmic scale from the fourth checkpoint size to the total dataset size. Note that if you try to call `seed_labels` outside of the first four checkpoints, you will get an error:

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
r.json()

For the 5th through 8th checkpoints, you may only query by id and submit. At this point we have 15 labels left in our budget for this checkpoint. If you query for less than the labels in the budget and submit, you will forgo the additional labels you could have received and the `budget_used` will automatically by increased to the budget of the checkpoint for which you submitted. To demonstrate, we will request 10 labels and then submit.

In [ ]:
images_to_be_labeled = get_random_labels_from_train_dataset(DATASETS_PATH, session_token, n=10)
images_to_be_labeled

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

query = {
    'example_ids': images_to_be_labeled
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
r.json()

Our `budget_left` is 5 and `budget_used` is 90, but if we submit now, our `budget_used` will automatically be increased to the budget of the checkpoint: `95`. 

In [ ]:
# Submit predictions
df = generate_random_predictions_on_test_set(test_imgs, current_dataset_classes)
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

# Probabilistic Predictions

The second way is to report the probabilities for each class by submitting a dataframe with the columns 'id' and a column for each class name. For instance, the following example is based on MNIST. The columns are the string names of the classes.

In [ ]:
def generate_random_probabilities_on_test_set(test_imgs: List[str], current_dataset_classes: List[str]) -> pd.DataFrame:
    """
    Generates a prediction dataframe for image classification with probabilities.
    
    example:
    
    ========  ====  ====  ====  ====  ====  ====  ====  ====  ====
    id        '1'   '2'   '3'   '4'   '5'   '6'   '7'   '8'   '9'
    ========  ====  ====  ====  ====  ====  ====  ====  ====  ====
    6831.png  0.01  0.09  0.0   0.25  0.65  0.0   0.0   0.0   0.0
    1186.png  0.15  0.0   0.20  0.25  0.05  0.35  0.0   0.0   0.0
    8149.png  0.80  0.10  0.0   0.05  0.0   0.05  0.0   0.0   0.0
    4773.png  0.0   0.7   0.0   0.15  0.15  0.0   0.0   0.0   0.0
    3752.png  0.0   0.10  0.0   0.0   0.0   0.9   0.0   0.0   0.0
    ========  ====  ====  ====  ====  ====  ====  ====  ====  ==== 
    """
    probabilities = []
    for _ in range(len(test_imgs)):
        a = np.random.random(size=len(current_dataset_classes))
        a /= a.sum()
        probabilities.append(a)
    df = pd.DataFrame(probabilities, columns=current_dataset_classes)
    df['id'] = test_imgs

    return df

In [ ]:
df = generate_random_probabilities_on_test_set(test_imgs, current_dataset_classes)

In [ ]:
df.head()

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

Since we have gone through 5 of the 8 checkpoints at this point, let's just finish out some additional dummy submissions until we do our 8th submission, finishing out the `pair_stage` = `base` and so we can move on to the `adaptation` phasae. 


In [ ]:
for _ in (range(2)):
    # Checkpoint ith submission
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
    

# We had 8 stages, we submitted 8 sets of predictions

## Notice when we query for session_status now, our `pair_stage` = `adaptation` instead of `base`. Also, we have a different set of `current_label_budget_stages`

Our example is reusing the `mnist` dataset, however, there will almost always be a different dataset between `base` and `adaptation`. This `mnist` -> `mnist` example is just showing how to use the api.

# Zero-shot Learning Workflow

For zero-shot learning, we can only use `get_seen_labels` to request labels for training images that belong to the seen classes. Calls to the `seed_labels` endpoint will result in an exception.

## Metrics
The optional ZSL task returns the accuracy, average per-class recall, ROC_AUC, and top-5 accuracy. These metrics will be calculated separately for the unseen classes, seen classes, as well as the overall dataset with all classes. The `submit_predictions` endpoint will return the following metrics in `response["Session_Status"]["checkpoint_scores"]`: 
<style type="text/css">
.tg  {border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:1px;color:#333;
  font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:1px;color:#333;
  font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">Metric</th>
    <th class="tg-0lax">Class splits</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax" rowspan="3">Accuracy (Top-1 and Top-5)</td>
    <td class="tg-0lax">All</td>
  </tr>
  <tr>
    <td class="tg-0lax">Seen</td>
  </tr>
  <tr>
    <td class="tg-0lax">Unseen</td>
  </tr>
  <tr>
    <td class="tg-0lax" rowspan="3">Average per-class recall</td>
    <td class="tg-0lax">All</td>
  </tr>
  <tr>
    <td class="tg-0lax">Seen</td>
  </tr>
  <tr>
    <td class="tg-0lax">Unseen</td>
  </tr>
  <tr>
    <td class="tg-0lax" rowspan="3">ROC_AUC</td>
    <td class="tg-0lax">All</td>
  </tr>
  <tr>
    <td class="tg-0lax">Seen</td>
  </tr>
  <tr>
    <td class="tg-0lax">Unseen</td>
  </tr>
</tbody>
</table>

In [1]:
import os
import random
import requests
import pandas as pd
from pprint import pformat

url = 'https://api-dev.lollllz.com'

# Base Headers
headers = {
    'user_secret': os.environ.get('TESTS_SECRET'),
    'govteam_secret': os.environ.get('GOVTEAM_SECRET')
}

# Create session
session_config = {
    "session_name": "zsl_test_session", "data_type": "sample",
    "task_id": "problem_test_zsl_2", "ZSL": True
}
session_token = requests.post(
    f"{url}/auth/create_session", json=session_config, headers=headers
).json()["session_token"]

session_headers = {"session_token": session_token, **headers}

session_status = requests.get(
    f"{url}/session_status",
    headers=session_headers
).json()["Session_Status"]
print(f"Session status:\n{pformat(session_status, depth=3, width=150)}")

# Add all labeled files to a list for use in training
classes = session_status["current_dataset"]["classes"]
labeled_files = {class_: set() for class_ in classes}
labeled_examples = requests.get(f"{url}/get_seen_labels", headers=session_headers).json()["Labels"]
for example in labeled_examples:
    class_name, file = example["class"], example["id"]
    labeled_files[class_name].add(file)

# Obtain ZSL descriptions (optional)
dataset_metadata = session_status["current_dataset"]
zsl_descriptions = dataset_metadata["zsl_description"]
unseen_classes = dataset_metadata.get(
    "unseen_classes", sorted(zsl_descriptions.keys())
)
seen_classes = dataset_metadata.get(
    "seen_classes", sorted(set(dataset_metadata["classes"]) - set(unseen_classes))
)

# Train model using labeled_files (Example):
# model = torch.hub.load("pytorch/vision:0.10.0", "resnet18", pretrained=True)
# train_loader = get_train_loader(labeled_files, batch_size=32)
# optimizer = torch.optim.Adam(model.parameters(), lr=4e-3)
# loss = torch.nn.CrossEntropyLoss()
# train(model, num_epochs, train_loader, optimizer, loss)

# To train model with TRAINING images from unseen classes, can do something like this:
unseen_files = requests.get(f"{url}/get_unseen_ids", headers=session_headers).json()["ids"]

def model(input_files):  # dummy random model
    return [str(random.choice(classes)) for _ in input_files]

# Use trained model to obtain predictions
## Obtain test files first
data_root = os.path.join(
    os.path.expanduser("~"), "Documents", "LWLL", "lwll_datasets", "development"
)
dataset = session_status["current_dataset"]["name"]
test_folder = os.path.join(
    data_root, dataset, f"{dataset}_{session_config['data_type']}", "test"
)
test_files = os.listdir(test_folder)

# Optional: Train standard ZSL model and submit standard ZSL predictions
def standard_zsl_model(input_files):  # dummy standard zsl model
    return [str(random.choice(unseen_classes)) for _ in input_files]
std_pred_df = pd.DataFrame({'id': test_files, 'class': standard_zsl_model(test_files)})
## Submit standard ZSL prediction (predictions on images from seen classes are ignored)
response = requests.post(
    f"{url}/submit_standard_zsl_predictions",
    json={"predictions": std_pred_df.to_dict()},
    headers=session_headers,
).json()

# Submit generalized ZSL predictions
preds = model(test_files)
pred_df = pd.DataFrame({'id': test_files, 'class': preds})
response = requests.post(
    f"{url}/submit_predictions",
    json={"predictions": pred_df.to_dict()},
    headers=session_headers,
).json()
session_status_post_submission = response["Session_Status"] 
# remove "zsl_description" from session status to avoid printing it
import copy
_s = copy.deepcopy(session_status_post_submission)
_s["current_dataset"].pop("zsl_description")
print(
    f"\nSession status after submission:\n"
    f"=================================\n"
    f"{pformat(_s, compact=True)}"
)

Session status:
{'ZSL': True,
 'active': 'In Progress',
 'budget_left_until_checkpoint': 0,
 'budget_used': 0,
 'checkpoint_scores': [],
 'current_dataset': {'classes': ['forest',
                                 'buildings',
                                 'river',
                                 'mobile_home_park',
                                 'harbor',
                                 'golf_course',
                                 'agricultural',
                                 'runway',
                                 'baseball_diamond',
                                 'overpass',
                                 'chaparral',
                                 'tennis_court',
                                 'intersection',
                                 'airplane',
                                 'parking_lot',
                                 'sparse_residential',
                                 'medium_residential',
                                 'dense_residential

# Unsupervised Domain Adaptation

Notice that now we have switched from the base dataset to the adaptation dataset. At this point, you may optionally submit predictions for unsupervised domain adaptation (UDA) before requesting any labels on the adaptation dataset using the endpoint `submit_UDA_predictions`.  There are two fields in the `task_metadata`, which we queried at the beginning of the notebook, that provide information about class overlap: `uda_base_to_adapt_overlap_ratio` and `uda_adapt_to_base_overlap_ratio`

In [ ]:
r = requests.post(f"{url}/submit_UDA_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

# *Gotchas and points to note

Before we completely finish our session, let's talk about some gotchas and requirements of these endpoints.

**1.)** When you query for labels, if you ask for more labels than your `budget_left_until_checkpoint` in your session, it will be truncated and you will only recieve the number of labels up until that number.

**2.)** The next gotcha is when we submit our predictions, YOU MUST submit all of the `id`'s and `label`'s. If you do not, you will encounter an error. 

In [ ]:
bad_df = pd.DataFrame({'id':['14647.png', '33864.png'], 'class':['4', '3']})

r = requests.post(f"{url}/submit_predictions", json={'predictions': bad_df.to_dict()}, headers=headers)
r.json()

# Wrap up

To finish out our `session`, let's just submit valid final predictions and then notice how on the last one, our `active` flag turns to `False`. This means we have successfully finished!!

In [ ]:
for i in range(8):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
    print(json.dumps(r.json(), indent=2))

We have now hit all the checkpoints and finished the session
You'll notice after we finish a session we now have access to see our metric scores at the end and our `active` flag transitions from `In Progress` to `Complete`

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/session_status", headers=headers)
r.json()

At this point in the automated system, you should go on to the next problem you see from the `/list_tasks` endpoint and go through until you are finished.

## We now do a walk through of the second example problem type

The second problem type is the `object_detection` one. The flow is the same, except that you will notice that we have different label format and scoring metrics.


In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/task_metadata/problem_test_obj_detection", headers=headers)
r.json()

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
data_type = 'sample' # can either be `sample` or `full`

r = requests.post(f"{url}/auth/create_session", json={'session_name': 'testing', 'data_type': data_type, 
                                                      'task_id': 'problem_test_obj_detection'}, 
                  headers=headers)
r.json()

In [ ]:
session_token = r.json()['session_token']

Exactly as before we need to follow the steps of:

For EACH `base` and `adaptation` phase:

For the first 4 checkpoints
- Call `/seed_labels` *or* `/query_labels`
- Submit predictions


Then the last 4 checkpoints
- Query for additional labels to `/query_labels`
- Submit predictions

For the sake of just closing out our session since this example is to demonstrate the  format for object detection problems, we will loop through the `seed_labels` and  predictions. First we show the format of predictions:

In [ ]:
test_imgs, current_dataset_classes = get_test_images_and_classes(DATASETS_PATH, session_token)

In [ ]:
def generate_random_predictions_on_test_set_obj_detection(test_imgs: List[str], current_dataset_classes: List[str]) -> pd.DataFrame:
    """
    Generates a prediction dataframe for image classification based on random sampling from our available classes
    """
    
    # We just use random labels for example. Our labels have to have a bounding box, confidence and class for object detection
    # bounding boxes are defined as '<xmin>, <ymin>, <xmax>, <ymax>''
    # This would be your inferences filling this DataFrame though.
    rand_lbls = ['20, 20, 80, 80' for _ in range(len(test_imgs))]
    conf = [0.95 for _ in range(len(test_imgs))]
    classes = [current_dataset_classes[0] for _ in range(len(test_imgs))]
    df = pd.DataFrame({'id': test_imgs, 'bbox': rand_lbls, 'confidence': conf, 'class': classes})
    return df

In [ ]:
df = generate_random_predictions_on_test_set_obj_detection(test_imgs, current_dataset_classes)

In [ ]:
df.head()

We submit these as a dictionary in this format:

In [ ]:
df.head().to_dict()

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
r.json()

In [ ]:
for _ in range(4):
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

    r = requests.get(f"{url}/seed_labels", headers=headers)
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

At the last 4 checkpoints, you can query by id, but we will just submit. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
# Base dataset predictions
for _ in range(4):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

Get the test images and random predictions for `adaptation` dataset

In [ ]:
test_imgs, current_dataset_classes = get_test_images_and_classes(DATASETS_PATH, session_token)
df = generate_random_predictions_on_test_set_obj_detection(test_imgs, current_dataset_classes)

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
# Adaptation dataset predictions
for _ in range(8):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)

Now verify that our session is over.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

# Video Classification

As in image classification and object detection, for the first four checkpoints you can choose to request seed labels or query by id. For the last four checkpoints, you can only query by id. 

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
data_type = 'sample' # can either be `sample` or `full`

r = requests.post(f"{url}/auth/create_session", json={'session_name': 'testing', 'data_type': data_type, 'task_id': 'problem_test_video_classification'}, headers=headers)
r.json()
session_token = r.json()['session_token']

Let's get our first set of seed labels and look at the output. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
print(json.dumps(r.json(), indent = 4))

For video classification, you are given the start and end frame number for the action. To get all of the frames for the action, simply enumerate between this range and add `.jpg`. Frames will always be provided in `.jpg` form. For example, to get all the frames associated with the first label:

In [ ]:
labels = r.json()["Labels"]
action_frames = [str(i)+'.jpg' for i in range(labels[0]['start_frame'], labels[0]['end_frame'] +1)]
action_frames

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

We need to submit predictions before we can query for seed labels again. Here we define some helper functions to retreive the video train data and test ids.

You will notice that in the test set metadata, you are given the start and end frames for each segment corresponding to one action. These are given so that you have the boundaries on which to perform inference, as we do not consider determining these boundaries to be part of the image classification task. THIS DATA IS TO BE USED ONLY FOR INFERENCE. ANY USE OF THE TEST DATA FOR TRAINING WILL BE CONSIDERED CHEATING. 

In [ ]:
def get_test_images_and_classes_vid(dataset_path: Path, session_token: str, data_type: str='sample') -> Tuple[List[str],List[str]]:
    """
    Helper method to dynamically get the test labels and give us the possible classes that can be submitted
    for the current dataset
    
    Params
    ------
    
    dataset_path : Path
        The path to the `development` dataset downloads
    
    session_token : str
        Your current session token so that we can look up the current session metadata
    
    data_type: str
        Indicates whether you are using the `sample` or `full` dataset. 
    Returns
    -------
    
    Tuple[List[str], List[str]]
        The list of test image ids needed to submit a prediction and the list of class names that you can predict against
    """
    # Then we can just reference our current metadata to get our dataset name and use that in the path
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    r = requests.get(f"{url}/session_status", headers=headers)
    current_dataset = r.json()['Session_Status']['current_dataset']
    current_dataset_name = current_dataset['name']
    current_dataset_classes = current_dataset['classes']

    test_meta = pd.read_feather(dataset_path.joinpath(f"{current_dataset_name}/labels_{data_type}/meta_test.feather"))
    test_ids = test_meta['id'].tolist()
    return test_ids, current_dataset_classes


In [ ]:
test_ids, current_dataset_classes = get_test_images_and_classes_vid(dataset_path=DATASETS_PATH, 
                                                                    session_token=session_token, 
                                                                    data_type=data_type)
df = generate_random_predictions_on_test_set(test_ids, current_dataset_classes)

In [ ]:
df.head()

Submit our predictions and check the response

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

Now we will request seed labels and submit three more times.

In [ ]:
for i in range(3):
    print(i+1)
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    r = requests.get(f"{url}/seed_labels", headers=headers)
    # get session metadata
    r = requests.get(f"{url}/session_status", headers=headers)
    budget_used = r.json()['Session_Status']['budget_used']
    budget_left = r.json()['Session_Status']['budget_left_until_checkpoint']
    print(f"Got seed labels. Budget used: {budget_used} Budget left: {budget_left}")
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
print(json.dumps(r.json(), indent=4))

In [ ]:
 r = requests.get(f"{url}/session_status", headers=headers)

In [ ]:
r.json()

### Querying for video labels
For the last four checkpoints, we can query for labels by id for a video segment. The segments are provided in the `meta_train.feather` file when you download the dataset. Notice that our helper function is changed slightly to reflect this. Instead of iterating over all the frames in the `train` folder, we load the `meta_train.feather` file to look up the video segment `id`s. You can query up until you reach your label budget. The format of the response is the same as for seed labels. In the following example, we will query for two ids. 

In [ ]:
def get_random_labels_from_train_dataset_vid(dataset_path: Path, session_token: str, n: int=None, data_type: str='sample') -> List[str]:
    """
    Helper function to get random `n` video segment ids from our train dataset to request labels for
    from the api
    
    Params
    ------
    
    dataset_path : Path
        The path to the `development` dataset downloads
    
    session_token : str
        Your current session token so that we can look up the current session metadata
    data_type: str
        Indicates whether you are using the `sample` or `full` size dataset
    
    Returns
    -------
    
    List[str]
        A list of n unique image ids for the current session dataset
        
    """
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    r = requests.get(f"{url}/session_status", headers=headers)
    current_dataset = r.json()['Session_Status']['current_dataset']
    current_dataset_name = current_dataset['name']
    budget_left = r.json()['Session_Status']['budget_left_until_checkpoint'] 
    if not n:
        n = budget_left
        
        print(f"budget_left is {budget_left}")
      
    meta_train_path = dataset_path.joinpath(f"{current_dataset_name}/labels_{data_type}/meta_train.feather")
    meta_train = pd.read_feather(meta_train_path)
    random_ids = meta_train['id'].sample(n=n).tolist()
    return random_ids

In [ ]:
segments_to_be_labeled = get_random_labels_from_train_dataset_vid(DATASETS_PATH, session_token, n=77)
query = {
    'example_ids': segments_to_be_labeled
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
print(len(r.json()['Labels']))

We will now submit for the last 4 checkpoints to advance to the adaptation stage, but you could continue to query for labels throughout these checkpoints. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

for _ in range(4):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

### Optionally submit predictions for unsupervised domain adaptation

In [ ]:
r = requests.post(f"{url}/submit_UDA_predictions", json={'predictions': df.to_dict()}, headers=headers)
r.json()

For the sake of closing out the session, we will now loop through the adaptation stage. As in the other problem types, for the adaptation stage in the first four checkpoints you may get seed labels or query for image labels, and submit predictions for each checkpoint. In the last four checkpoints you may query for image labels and submit predictions for each checkpoint. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
# Adaptation dataset predictions
for _ in range(8):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': df.to_dict()}, headers=headers)

In [ ]:
r.json()

## Machine Translation

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/task_metadata/06023f86-a66b-4b2c-8b8b-951f5edd0f22", headers=headers)
r.json()

In [ ]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
data_type = 'full' # can either be `sample` or `full`

r = requests.post(f"{url}/auth/create_session", json={'session_name': 'testing', 'data_type': data_type, 'task_id': '06023f86-a66b-4b2c-8b8b-951f5edd0f22'}, headers=headers)
r.json()

In [ ]:
session_token = r.json()['session_token']

There is no concept of seed labels in machine translation, we just go through the 8 checkpoints in an active learning workflow. Note that if you try to call the `seed_labels` endpoint, you will receive a warning and `None` will be returned

In [ ]:
def get_train_data_mt(dataset_path: Path, session_token: str) -> List[str]:
    """
    Helper method to dynamically get the test labels and give us the possible classes that can be submitted
    for the current dataset
    
    Params
    ------
    
    dataset_path : Path
        The path to the `development` dataset downloads
    
    session_token : str
        Your current session token so that we can look up the current session metadata
    
    Returns
    -------
    
    pd.DataFrame
        The DataFrame on which you can make queries against
    """
    # Then we can just reference our current metadata to get our dataset name and use that in the path
    headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
    r = requests.get(f"{url}/session_status", headers=headers)
    current_dataset = r.json()['Session_Status']['current_dataset']
    current_dataset_name = current_dataset['name']

    test_df = pd.read_feather(str(dataset_path.joinpath(f"{current_dataset_name}/{current_dataset_name}_{data_type}/train_data.feather")))
    return test_df

def get_test_data_mt(dataset_path: Path, session_token: str) -> List[str]:
    """
    Helper method to dynamically get the test labels and give us the possible classes that can be submitted
    for the current dataset
    
    Params
    ------
    
    dataset_path : Path
        The path to the `development` dataset downloads
    
    session_token : str
        Your current session token so that we can look up the current session metadata
    
    Returns
    -------
    
    pd.DataFrame
        The DataFrame on which you must make predictions from a 'source' column
    """
    # Then we can just reference our current metadata to get our dataset name and use that in the path
    headers = {'user_secret': secret, 'session_token': session_token}
    r = requests.get(f"{url}/session_status", headers=headers)
    current_dataset = r.json()['Session_Status']['current_dataset']
    current_dataset_name = current_dataset['name']
    
    _path = str(dataset_path.joinpath(f"{current_dataset_name}/{current_dataset_name}_{data_type}/test_data.feather"))
    test_df = pd.read_feather(_path)
    return test_df

In [ ]:
train_df = get_train_data_mt(DATASETS_PATH, session_token)

In [ ]:
train_df.head()

In [ ]:
train_df.shape

Check our session status.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token}
r = requests.get(f"{url}/session_status", headers=headers)
r.json()

Here we will demonstrate what will happen if we try to request over our label budget. Our budget is `5000` characters, so first we will request 50 labels.

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

query = {
    'example_ids': [str(i) for i in range(50)]
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
print(len(r.json()['Labels']))
#show the format of 1 label
print(r.json()['Labels'][0])      

Check our session status. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/session_status", headers=headers)
r.json()

Our `budget_left` is `296` characters. Let's try requesting an additional 10 labels. 

In [ ]:
query = {
    'example_ids': [str(i) for i in range(50, 60)]
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
len(r.json()['Labels'])

This put us over our label budget, so we got fewer than the 10 sentences that we requested. 

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/session_status", headers=headers)
r.json()

Now make a prediction submission

In [ ]:
test_df = get_test_data_mt(DATASETS_PATH, session_token)

In [ ]:
test_df.head()

In [ ]:
def generate_random_predictions_on_test_set_mt(test_df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates a prediction dataframe for machine translation with fake prediction data
    """
    
    # We make predictions and want a DataFrame with the columns
    # 'id' and 'text'
    pred = 'The quick brown fox jumps over the lazy dog'
    pred_list = [pred for _ in range(len(test_df))]
    df = pd.DataFrame({'id': test_df['id'].tolist(), 'text': pred_list})
    return df

In [ ]:
pred_df = generate_random_predictions_on_test_set_mt(test_df)

In [ ]:
pred_df.head()

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.post(f"{url}/submit_predictions", json={'predictions': pred_df.to_dict()}, headers=headers)
r.json()

We successfully submitted a prediction file, let's go through the rest of the 7 checkpoints submitting our dummy prediction file until we switch over to adaptation

In [ ]:
for _ in range(7):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': pred_df.to_dict()}, headers=headers)

In [ ]:
r.json()

And now we see the dataset has flipped over to the `adaptation` dataset, which is `ted_talks`

Let's get our dummy test labels for this new dataset and finish out our session by submitting the last 8 dummy checkpoints.

In [ ]:
test_df = get_test_data_mt(DATASETS_PATH, session_token)
pred_df = generate_random_predictions_on_test_set_mt(test_df)

In [ ]:
test_df.head()

In [ ]:
len(train_df)

In [ ]:
len(test_df)

In [ ]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

query = {
    'example_ids': [str(i) for i in range(5000)]
}

r = requests.post(f"{url}/query_labels", json=query, headers=headers)
print(len(r.json()['Labels']))

In [ ]:
pred_df.head()

In [ ]:
for _ in range(8):
    r = requests.post(f"{url}/submit_predictions", json={'predictions': pred_df.to_dict()}, headers=headers)

In [ ]:
r.json()

And now we can see that we successfully finished the task!

*Initially written - 11/15/19 MH*

*Update for cleanliness - 11/21/19 MH*

*Added Object Detection support - 12/16/19 MH*

*Added Machine Translation support - 12/22/19 MH*

*Updated predictions naming schema form `label` to `class` for `image_classification` and `label` to `bbox` for `object_detection` - 1/28/20 MH*

*Changed `get_session_token` to `create_session`, performers can now customize the `session_name` - 2/10/20 AY*

*Updated to use example ids from new mnist base/adaptation splits - 04/01/20 AD*

*Removed machine translation, updated submission times for new numbers of label budgets - 04/29/20 AD*

*Verified updated task metadata, included some automation around getting test images MH*

*Added machine translation back in with the implementation of the secondary backend - 5/17/20 MH*

*Added example of passing govteam_secret through request headers for eval time - 7/25/20 MH*

*Removed `secondary_seed_labels` and updated `seed_labels` examples to reflect the eval2 budget scheme. `seed_labels` can now be called for the first four checkpoints. - 02/02/20 AD*